Chandani_Gupta_Datahack

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

Loading Datasets

In [3]:
training_features=pd.read_csv('training_set_features.csv')
test_features=pd.read_csv('test_set_features.csv')
training_labels=pd.read_csv('training_set_labels.csv')
submission_format=pd.read_csv('submission_format.csv')

In [4]:
training_features

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


Combining training features with training labels

In [5]:
data=training_features.merge(training_labels, on='respondent_id')

In [6]:
print(data.dtypes)

respondent_id                    int64
xyz_concern                    float64
xyz_knowledge                  float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_xyz                float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_xyz_vacc_effective     float64
opinion_xyz_risk               float64
opinion_xyz_sick_from_vacc     float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                      

In [7]:
data.isnull().sum()

respondent_id                      0
xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [8]:
data.describe()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,xyz_vaccine,seasonal_vaccine
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000,26707.000000,26707.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583,0.212454,0.465608
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173,0.409052,0.498825
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000,0.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000,1.000000,1.000000


In [9]:
print("\nChecking for duplicates:")
duplicated_rows=pd.DataFrame(data[data.duplicated()])
print("Duplicates found:")
print(duplicated_rows)


Checking for duplicates:
Duplicates found:
Empty DataFrame
Columns: [respondent_id, xyz_concern, xyz_knowledge, behavioral_antiviral_meds, behavioral_avoidance, behavioral_face_mask, behavioral_wash_hands, behavioral_large_gatherings, behavioral_outside_home, behavioral_touch_face, doctor_recc_xyz, doctor_recc_seasonal, chronic_med_condition, child_under_6_months, health_worker, health_insurance, opinion_xyz_vacc_effective, opinion_xyz_risk, opinion_xyz_sick_from_vacc, opinion_seas_vacc_effective, opinion_seas_risk, opinion_seas_sick_from_vacc, age_group, education, race, sex, income_poverty, marital_status, rent_or_own, employment_status, hhs_geo_region, census_msa, household_adults, household_children, employment_industry, employment_occupation, xyz_vaccine, seasonal_vaccine]
Index: []

[0 rows x 38 columns]


Mapping ordinal features to their respective numerical values

In [38]:
ordinal_mapping={"xyz_concern":{0: "Not at all concerned", 1: "Not very concerned", 2: "Somewhat concerned", 3: "Very concerned"}, "xyz_knowledge":{0:"No knowledge", 1:"A little knowledge", 2:"A lot of knowledge"}, "opinion_xyz_vacc_effective":{0:"Not at all effective", 1:"Not very effective", 2:"Don't know", 3:"Somewhat effective", 4:"Very effective"}, "opinion_xyz_risk":{0:"Very Low", 1:"Somewhat low", 2:"Don't know", 3:"Somewhat high", 4:"Very high"}, "opinion_xyz_sick_from_vacc":{0:"Not at all worried", 1:"Not very worried", 2:"Don't know", 3:"Somewhat worried", 4:"Very worried"}, "opinion_seas_vacc_effective":{0:"Not at all effective", 1:"Not very effective", 2:"Don't know", 3:"Somewhat effective", 4:"Very effective"},  "opinion_seas_risk":{0:"Very Low", 1:"Somewhat low", 2:"Don't know", 3:"Somewhat high", 4:"Very high"}, "opinion_seas_sick_from_vacc":{0:"Not at all worried", 1:"Not very worried", 2:"Don't know", 3:"Somewhat worried", 4:"Very worried"}}

In [39]:
for feature, mapping in ordinal_mapping.items():data[feature]=data[feature].map(mapping)

Defining Categorical Features for OneHotEncoding

In [40]:
categorical_features=['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa','employment_industry','employment_occupation']

Defining numerical features that don't need encoding

In [41]:
numerical_features=['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds','behavioral_avoidance','behavioral_face_mask','behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home','behavioral_touch_face','doctor_recc_xyz','doctor_recc_seasonal','chronic_med_condition','child_under_6_months','health_worker','health_insurance','opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults', 'household_children']

Combining categorical and numerical features

Using Imputers for filling missing values

In [42]:
num_pipeline=Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
cat_pipeline=Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder())])

Defining ColumnTransformer to apply different preprocessing to different columns

In [43]:
preprocessor=ColumnTransformer(transformers=[('num', num_pipeline, numerical_features), ('cat', cat_pipeline, categorical_features)])

Combining categorical and numerical features

In [44]:
features=numerical_features+categorical_features

In [47]:
for feature in features:
    if data[feature].isnull().all():
        if data[feature].dtype=='object':
            data[feature].fillna('Unknown', inplace=True)
        else:
            data[feature].fillna(-1, inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_12652\797993894.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna('Unknown', inplace=True)


Defining target variable

In [36]:
y=data[['xyz_vaccine', 'seasonal_vaccine']]

Preprocessing the features

In [37]:
X=preprocessor.fit_transform(data[features])

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Unknown'

Initializing the MultiouputClassifier with RandomForest

In [20]:
forest=RandomForestClassifier(random_state=42)
multi_target_forest=MultiOutputClassifier(forest, n_jobs=-1)

Training the model using training data only

In [ ]:
multi_target_forest.fit(X,y)

Predicting Probabilities on actual test set

In [ ]:
X_test_actual=preprocessor.transform(test_features[features])
y_pred_proba_actual=multi_target_forest.predict_proba(X_test_actual)

Extracting probabilities

In [ ]:
xyz_proba_actual=y_pred_proba_actual[0][:,1]
seasonal_proba_actual=y_pred_proba_actual[1][:,1]

Calculate ROC AUC for each target variable

In [ ]:
roc_auc_xyz=roc_auc_score(y['xyz_vaccine'], xyz_proba_actual)
roc_auc_seasonal=roc_auc_score(y['seasonal_vaccine'], seasonal_proba_actual)
mean_roc_auc=(roc_auc_xyz+roc_auc_seasonal)/2

In [ ]:
print(f'ROC AUC for xyz_vaccine:{roc_auc_xyz}')
print(f'ROC AUC for seasonal_vaccine:{roc_auc_seasonal}')
print(f'Mean ROC AUC:{mean_roc_auc}')

Preparing Submission File probabilities

In [ ]:
submission=submission_format.copy()
submission['xyz_vaccine']=xyz_proba_actual
submission['seasonal_vaccine']=seasonal_proba_actual
submission.to_csv('submission_format.csv',index=False)